In [5]:
from sqlalchemy import create_engine
import numpy as np
import pandas as pd

In [7]:
conn1 = create_engine('impala://172.24.20.123:21050')
sql = '''
SELECT pp.product_code'商品ID', pp.product_name'商品名称', pp.packing'规格', pp.manufacturer'厂家',
    tgzh.pd_view_user_num'访客数', tgzh.search_click_user_cnt'搜索点击用户数', tgzh.pd_view_purchase_rate'加购转化率', tgzh.pd_pay_rate'支付转化率',
    tgzh.sales_per_uv'人均uv价值', tgzh.trade_index'交易指数(天猫)', tgzh.avg_price_index'PI指数',
    tgdx.vi_cnt'营销访客_短信',tgdx.ac_cnt'营销下单_短信',
    tgdx.ac_cnt_new'营销新客_短信',tgdx.pr_amount_new'营销新客下单额_短信',
    l1.fks'营销访客_推广', l1.xd'营销下单_推广', l1.lxrs'营销新客_推广', l1.lxje'营销新客下单额_推广', l1.pjrs'评价人数', l1.pjfs'评价分数', 
    pf7.product_fill_rate '订单满足率', pf7.duration'发货时长(h)',
    dyxs.ac_cnt'客户数', dyxs.plor_amount'销售额', dyxs.plor_gross'毛利额'
from jkbd.product_parquet pp 
left join default.lin_product_index_190827 tgzh on pp.product_code=tgzh.product_code
left join (
    SELECT product_code, avg(click_accnt_24h) vi_cnt,
        sum(accounts_cnt_24h_product) ac_cnt, 
        sum(if(is_new = 1,accounts_cnt_24h_product,0)) ac_cnt_new,
        sum(if(is_new = 1,order_amount_24h_product,0)) pr_amount_new
    from default.sms_promotion_info
    where date_trunc('month', plan_time) = '2019-07-01 00:00:00'
    GROUP BY product_code
) tgdx on pp.product_code=tgdx.product_code
left join (
    SELECT class_segment, large_class, product_code,
        count(DISTINCT fedf.account_id) ac_cnt,
        sum(product_amount) plor_amount, 
        sum(product_amount-product_cost) plor_gross
    from jkbd.fact_export_detail_full fedf
    -- 必备条件
    where fedf.EL_RECORD_STATE = 2 
        and fedf.EF_RECORD_STATE = 2 
        and fedf.order_type not in ('门店', '批零一体化', '批发')
        and fedf.export_form_type not in ('退货单')
        and fedf.form_rank = 1
        and fedf.is_group = 0
        and fedf.is_gift = '非赠品'
        and fedf.product_code not in (857932, 857934)
        -- 业务条件
        and upper(fedf.sls_origin) rlike 'APP'
        and date_trunc('month', order_time) = '2019-07-01 00:00:00'
    GROUP BY class_segment, large_class, product_code
) dyxs on pp.product_code = dyxs.product_code
left join default.product_fd_7 pf7 on pp.product_code=pf7.product_code 
left join default.lu_190828 l1 on cast(pp.product_code as string)=l1.productid
where pp.product_status_type=2
    and pp.product_code not between 857032 and 857935
    and pp.is_gifts = 0
'''
df1 = pd.read_sql(sql, conn1)

In [ ]:
df1.info()

In [ ]:
df1=df1.fillna(0)

In [30]:
df2 = df1.drop({'商品名称','规格','厂家'}, axis=1)

In [31]:
df2.head()

,商品id,访客数,搜索点击用户数,加购转化率,支付转化率,人均uv价值,交易指数(天猫),pi指数,营销访客_短信,营销下单_短信,...,营销下单_推广,营销新客_推广,营销新客下单额_推广,评价人数,评价分数,订单满足率,发货时长(h),客户数,销售额,毛利额
0,838757,3315.0,3194.0,0.008748,0.69697,0.211246,0.0,0.0,0.0,0.0,...,10.0,5.0,269.5,0.0,0.0,0.666667,48.066667,38.0,2395.5576,750.5576
1,157879,134.0,124.0,0.000000,0.00000,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0000,0.0000
2,8115,26.0,25.0,0.000000,0.00000,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0000,0.0000
3,8902,0.0,0.0,0.000000,0.00000,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0000,0.0000
4,8943,27.0,26.0,0.000000,0.00000,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0000,0.0000


In [51]:
df2.访客数.max(), df2.访客数.min()

(22481.0, 0.0)

In [52]:
print(3315*100/22481)

14.745785329834082


In [39]:
df3 = df2.访客数
df3.head()

0    3315.0
1     134.0
2      26.0
3       0.0
4      27.0
Name: 访客数, dtype: float64

In [49]:
df2['访客分数']=100/(df2.访客数.max()-df2.访客数.min())*(df2.访客数-df2.访客数.min())

In [53]:
df2.head()

,商品id,访客数,搜索点击用户数,加购转化率,支付转化率,人均uv价值,交易指数(天猫),pi指数,营销访客_短信,营销下单_短信,...,营销新客_推广,营销新客下单额_推广,评价人数,评价分数,订单满足率,发货时长(h),客户数,销售额,毛利额,访客分数
0,838757,3315.0,3194.0,0.008748,0.69697,0.211246,0.0,0.0,0.0,0.0,...,5.0,269.5,0.0,0.0,0.666667,48.066667,38.0,2395.5576,750.5576,14.745785
1,157879,134.0,124.0,0.000000,0.00000,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0000,0.0000,0.596059
2,8115,26.0,25.0,0.000000,0.00000,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0000,0.0000,0.115653
3,8902,0.0,0.0,0.000000,0.00000,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0000,0.0000,0.000000
4,8943,27.0,26.0,0.000000,0.00000,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0000,0.0000,0.120101


In [59]:
df3 = df2.iloc[:,1:].apply(lambda x : (x - np.min(x))/(np.max(x)-np.min(x)))
df3.head()

,访客数,搜索点击用户数,加购转化率,支付转化率,人均uv价值,交易指数(天猫),pi指数,营销访客_短信,营销下单_短信,营销新客_短信,...,营销新客_推广,营销新客下单额_推广,评价人数,评价分数,订单满足率,发货时长(h),客户数,销售额,毛利额,访客分数
0,0.147458,0.390321,0.008748,0.69697,0.000507,0.0,0.0,0.0,0.0,0.0,...,0.003765,0.002537,0.0,0.0,0.666667,0.036861,0.013447,0.002163,0.056643,0.147458
1,0.005961,0.015153,0.000000,0.00000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.054882,0.005961
2,0.001157,0.003055,0.000000,0.00000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.054882,0.001157
3,0.000000,0.000000,0.000000,0.00000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.054882,0.000000
4,0.001201,0.003177,0.000000,0.00000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.054882,0.001201


In [62]:
x = 0
0 if x==0 else 95/x if x<=95 else x/95

0

In [75]:
df4 = df2.loc[:,'pi指数'].apply(lambda x: 0 if x==0 else 95/x if x<=95 else x/95)
print(df4.max(), df4.min())

2.1052631578947367 0.0


In [76]:
df4.describe()

count    16306.000000
mean         0.494812
std          0.590153
min          0.000000
25%          0.000000
50%          0.000000
75%          1.061865
max          2.105263
Name: pi指数, dtype: float64

In [77]:
df41 = pd.DataFrame(df4)
df41.describe()

,pi指数
count,16306.000000
mean,0.494812
std,0.590153
min,0.000000
25%,0.000000
50%,0.000000
75%,1.061865
max,2.105263


In [101]:
df5['pi分'] = df41.apply(lambda x: (x - np.min(x)/(np.max(x) - np.min(x))))
df51.describe()

,pi指数,pi分
count,16306.000000,16306.000000
mean,0.494812,0.494812
std,0.590153,0.590153
min,0.000000,0.000000
25%,0.000000,0.000000
50%,0.000000,0.000000
75%,1.061865,1.061865
max,2.105263,2.105263


In [102]:
dfm = pd.merge(df3, df51['pi分'], left_index=True, right_index=True)

In [103]:
dfm.columns

Index(['访客数', '搜索点击用户数', '加购转化率', '支付转化率', '人均uv价值', '交易指数(天猫)', 'pi指数',
       '营销访客_短信', '营销下单_短信', '营销新客_短信', '营销新客下单额_短信', '营销访客_推广', '营销下单_推广',
       '营销新客_推广', '营销新客下单额_推广', '评价人数', '评价分数', '订单满足率', '发货时长(h)', '客户数',
       '销售额', '毛利额', '访客分数', 'pi分'],
      dtype='object')

In [109]:
dfm.drop(['pi指数'], axis=1)

,访客数,搜索点击用户数,加购转化率,支付转化率,人均uv价值,交易指数(天猫),营销访客_短信,营销下单_短信,营销新客_短信,营销新客下单额_短信,...,营销新客下单额_推广,评价人数,评价分数,订单满足率,发货时长(h),客户数,销售额,毛利额,访客分数,pi分
0,0.147458,0.390321,0.008748,0.696970,0.000507,0.000000,0.0,0.0,0.0,0.0,...,0.002537,0.000000,0.0,0.666667,0.036861,0.013447,0.002163,0.056643,0.147458,0.000000
1,0.005961,0.015153,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.054882,0.005961,0.000000
2,0.001157,0.003055,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.054882,0.001157,0.000000
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.054882,0.000000,0.000000
4,0.001201,0.003177,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.054882,0.001201,0.000000
5,0.000044,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.054882,0.000044,0.000000
6,0.000534,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.054882,0.000534,0.000000
7,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.054882,0.000000,0.000000
8,0.000311,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.054882,0.000311,0.000000
9,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.054882,0.000000,0.000000


In [110]:
dfn = pd.merge(df2.商品id, dfm, left_index=True, right_index=True)

In [111]:
dfn.head()

,商品id,访客数,搜索点击用户数,加购转化率,支付转化率,人均uv价值,交易指数(天猫),pi指数,营销访客_短信,营销下单_短信,...,营销新客下单额_推广,评价人数,评价分数,订单满足率,发货时长(h),客户数,销售额,毛利额,访客分数,pi分
0,838757,0.147458,0.390321,0.008748,0.69697,0.000507,0.0,0.0,0.0,0.0,...,0.002537,0.0,0.0,0.666667,0.036861,0.013447,0.002163,0.056643,0.147458,0.0
1,157879,0.005961,0.015153,0.000000,0.00000,0.000000,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.054882,0.005961,0.0
2,8115,0.001157,0.003055,0.000000,0.00000,0.000000,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.054882,0.001157,0.0
3,8902,0.000000,0.000000,0.000000,0.00000,0.000000,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.054882,0.000000,0.0
4,8943,0.001201,0.003177,0.000000,0.00000,0.000000,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.054882,0.001201,0.0


In [112]:
import xgboost as xgb
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
import csv

In [121]:
x, y = dfn.iloc[:,1:-5], dfn.销售额

In [127]:
df2=df2.set_index('商品id')

In [128]:
df2.head()

,访客数,搜索点击用户数,加购转化率,支付转化率,人均uv价值,交易指数(天猫),pi指数,营销访客_短信,营销下单_短信,营销新客_短信,...,营销新客_推广,营销新客下单额_推广,评价人数,评价分数,订单满足率,发货时长(h),客户数,销售额,毛利额,访客分数
商品id,,,,,,,,,,,,,,,,,,,,,
838757,3315.0,3194.0,0.008748,0.69697,0.211246,0.0,0.0,0.0,0.0,0.0,...,5.0,269.5,0.0,0.0,0.666667,48.066667,38.0,2395.5576,750.5576,14.745785
157879,134.0,124.0,0.000000,0.00000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0000,0.0000,0.596059
8115,26.0,25.0,0.000000,0.00000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0000,0.0000,0.115653
8902,0.0,0.0,0.000000,0.00000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0000,0.0000,0.000000
8943,27.0,26.0,0.000000,0.00000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0000,0.0000,0.120101


In [129]:
x, y = df2.iloc[:, :-4], df2.销售额
x.head()

,访客数,搜索点击用户数,加购转化率,支付转化率,人均uv价值,交易指数(天猫),pi指数,营销访客_短信,营销下单_短信,营销新客_短信,营销新客下单额_短信,营销访客_推广,营销下单_推广,营销新客_推广,营销新客下单额_推广,评价人数,评价分数,订单满足率,发货时长(h)
商品id,,,,,,,,,,,,,,,,,,,
838757,3315.0,3194.0,0.008748,0.69697,0.211246,0.0,0.0,0.0,0.0,0.0,0.0,2868.0,10.0,5.0,269.5,0.0,0.0,0.666667,48.066667
157879,134.0,124.0,0.000000,0.00000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,100.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
8115,26.0,25.0,0.000000,0.00000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,19.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
8902,0.0,0.0,0.000000,0.00000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
8943,27.0,26.0,0.000000,0.00000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,16.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000


In [134]:
from sklearn import preprocessing


In [135]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.5, random_state=1)

lab_enc = preprocessing.LabelEncoder()
y_train = lab_enc.fit_transform(y_train)

lr = LogisticRegression(penalty='l2')
lr.fit(x_train, y_train)
y_hat = lr.predict(x_test)


D:\Anaconda37\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
D:\Anaconda37\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
D:\Anaconda37\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


NameError: name 'show_accuracy' is not defined

In [ ]:
print(lr)

In [ ]:
x, y = load_data('12.Titanic.train.csv', True)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.5, random_state=1)

lr = LogisticRegression(penalty='l2')
lr.fit(x_train, y_train)
y_hat = lr.predict(x_test)
lr_rate = show_accuracy(y_hat, y_test, 'Logistic回归 ')
# write_result(lr, 1)

rfc = RandomForestClassifier(n_estimators=100)
rfc.fit(x_train, y_train)
y_hat = rfc.predict(x_test)
rfc_rate = show_accuracy(y_hat, y_test, '随机森林 ')
# write_result(rfc, 2)

# XGBoost
data_train = xgb.DMatrix(x_train, label=y_train)
data_test = xgb.DMatrix(x_test, label=y_test)
watch_list = [(data_test, 'eval'), (data_train, 'train')]
param = {'max_depth': 3, 'eta': 0.1, 'silent': 1, 'objective': 'binary:logistic'}
         # 'subsample': 1, 'alpha': 0, 'lambda': 0, 'min_child_weight': 1}
bst = xgb.train(param, data_train, num_boost_round=100, evals=watch_list)
y_hat = bst.predict(data_test)
# write_result(bst, 3)
y_hat[y_hat > 0.5] = 1
y_hat[~(y_hat > 0.5)] = 0
xgb_rate = show_accuracy(y_hat, y_test, 'XGBoost ')

print 'Logistic回归：%.3f%%' % lr_rate
print '随机森林：%.3f%%' % rfc_rate
print 'XGBoost：%.3f%%' % xgb_rate